# How I solve the problem
1. Search `FireExit` in the IFC file.
2. Figure out the relations between `IfcDoor` and `FireExit` property:

  ```
  #279 = IFCDOOR('Door')

  <-|

  #300 = IFCRELDEFINESBYPROPERTIES( (#279) , #287)

  |->

  #287 = IFCPROPERTYSET('Pset_DoorCommon')

  -> #297 = IFCPROPERTYSINGLEVALUE('FireExit'), IFCBOOLEAN(.F.)
  ```
3. Use ifcopenshell to retrieve the property, referred to [list all property set, property names and element types](https://community.osarch.org/discussion/507/list-property-set-property-names-and-element-types-that-have-this)
4. Read conditions from csv file, referred to the tutorial [07_import_export.ipynb](https://github.com/jakob-beetz/ifcopenshell-notebooks/blob/main/07_import_export.ipynb)

**Notice**: # match...case feature requires python>=3.10

In [ ]:
# if you run the notebook in local env, please install python-ifcopenshel first.
%pip install python-ifcopenshell

In [2]:
import ifcopenshell
ifcf = ifcopenshell.open("data/example.ifc")

In [9]:
# check conditions in csv file
import csv
# attributes that could access directly with get_info()
direct_attributes = ['OverallHeight', 'OverallWidth']
# check if condition is met or not
# match...case feature requires python>=3.10
def meet_condition(check_value, requirement, value):
    match requirement:
        case 'larger than':
            return float(check_value) > float(value)
        case 'smaller than':
            return float(check_value) < float(value)
        case 'one of':
            return check_value in eval(value)
        case _:
            return str(check_value) == value
    
#open the file
with open(r'./data/properties_check.csv') as checkfile:
    # create a reader, that reads the columns into a dictionary
    # using the first row as keys ("Element", "Attribute", "Requirement", "Value"
    reader = csv.DictReader(checkfile,delimiter=";")
    # iterate over all rows
    for row in reader:
        element_type = row['Element']
        attribute = row['Attribute']
        requirement = row['Requirement']
        value = row['Value']
        is_direct_attribute = attribute in direct_attributes
        print('condition:',element_type, attribute, requirement, value)
        # get all elements 
        for element in ifcf.by_type(row['Element']):
            if is_direct_attribute:
                # this attribute could be simple accessed through get_info()
                attribute_value=element.get_info()[row['Attribute']]
                if not meet_condition(attribute_value, requirement, value):
                    print(f"Element {element.Name} {element.GlobalId} violates the requirement with {attribute}({attribute_value}) to be {requirement} {value}")
            else:
                # this atribute should be accessed through IfcRelDefinesByProperties
                for ifcPropertySet in map(lambda relDefines: relDefines.RelatingPropertyDefinition, element.IsDefinedBy):
                    for ifcProperty in filter(lambda p: p.Name == attribute, ifcPropertySet.HasProperties):
                        attribute_value = ifcProperty.NominalValue.wrappedValue
                        if not meet_condition(attribute_value, requirement, value):
                            print(f"Element {element.Name} {element.GlobalId} violates the requirement with {requirement} of {attribute} to be {value}")


condition: IfcDoor FireExit existence True
Element Door 0BVXp9RpzCo9u3Swr1c2NJ violates the requirement with existence of FireExit to be True
Element Door 0ZsYz4_vL4VffFmw$51LvN violates the requirement with existence of FireExit to be True
Element Door 0K8IiQQDjCYhn9BmgsH91y violates the requirement with existence of FireExit to be True
Element Door 2pII$tkkD4xO_dAXPSy0l$ violates the requirement with existence of FireExit to be True
condition: IfcDoor OverallHeight larger than 1500
condition: IfcWindow OverallHeight smaller than 1300
Element Window 2ERKaCBbX0ffhfTC_SAHgu violates the requirement with OverallHeight(1400.0) to be smaller than 1300
Element Window 1rpCrFM5T4jP9eu4hn0bwv violates the requirement with OverallHeight(1400.0) to be smaller than 1300
Element Window 0geWL1GwL6AQY7OJO$Vx8b violates the requirement with OverallHeight(1400.0) to be smaller than 1300
Element Window 1QXCXoVJz0nAYZ1k6wiJl$ violates the requirement with OverallHeight(1400.0) to be smaller than 1300
El